# Transport Problem

Transport flow optimization involves finding the most efficient way to transport quantities of an homogenuous good through a transportation network having production plants and a specific demand of the market, minimizing the transport costs while meeting specific constraints. It can be formulated as a linear optimization problem.

## Problem Formulation
Having different plants with specific supply limits and customers with specific demands at different locations, find the optimal transport flow to minimize transportation costs. Only one good is considered here, the production costs are assumed to be the same for each plant, only the transportation costs are relevant, which are proportional to the distance between plant and customer. 



### Mathematical formulation of the Transport Problem

Sets:
$$I :  \text{set of all plants}$$
$$J :  \text{set of all customers}$$
<p style="text-align:center;">no set of goods needed, only one good considered here</p>

Variables:
$$x_{ij} : \text{transport quantities from i to j in units}$$

Parameters:
$$d_{ij} > 0: \text{distance between supplier i and customer j}$$
$$c_{ij} = f * d_{ij}: \text{transport cost per unit, proportional to the distance}$$

$$ a_i: \text{capacity of plant i in units, } \forall i \in I $$
$$ b_i: \text{demand of customer j in units, } \forall j \in J $$

  
Objective:  <p style="text-align:center;">minimize the total cost to transport all unit of goods from the suppliers to the customers</p>  

$$min_x z = \sum_{i \in I} \sum_{j \in J}c_{ij} x_{ij}$$
subject to:

$$ \begin{gather}  
 \sum_{j \in J} x_{ij} <= a_i, \forall i \in I  \\  
 \sum_{i \in I} x_{ij} >= b_j, \forall j \in J \\
 x_{ij} >= 0, \forall i \in I, \forall j \in J.
\end{gather} $$

explanation: 
1) Constraint to reflect the supply limit of plant i 
2) Constraint to satisfy the demand of customer j
3) Constraint to ensure non-negative transport quantities

In [85]:
import math
import pandas as pd
import random
from typing import NamedTuple, Self

NUMBER_OF_PLANTS = 5
NUMBER_OF_CUSTOMERS = 10


def calc_distance(node1: tuple, node2: tuple) -> float:
    return math.sqrt((node1[0] - node2[0]) ** 2 + (node1[1] - node2[1]) ** 2)

# some random data
plants = pd.DataFrame.from_dict({
    id+1: [(random.random()*500, random.random()*500), random.randint(0,500)]
    for id in range(NUMBER_OF_PLANTS)
}, orient = "index", columns=["coordinates", "supply_limit"])

customers = pd.DataFrame.from_dict({
    id+1: [(random.random()*500, random.random()*500), random.randint(0,NUMBER_OF_PLANTS*10)]
    for id in range(NUMBER_OF_CUSTOMERS)
}, orient = "index", columns=["coordinates", "demand"]) 


In [82]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

transport_costs_per_km_and_unit = 0.2 # EUR/km /unit

# Sets
model.plants = pyo.Set(initialize=list(plants.index))
model.customers = pyo.Set(initialize=list(customers.index))

# Decision Variable
model.x = pyo.Var(model.plants, model.customers, within=pyo.NonNegativeIntegers )

# Params
def init_costs(model, i, j):
    node_i = plants.loc[i, "coordinates"]
    node_j = customers.loc[j, "coordinates"]
    return transport_costs_per_km_and_unit * calc_distance(node_i, node_j)
model.costs_per_quantity = pyo.Param(model.plants, model.customers, initialize=init_costs)

model.a = pyo.Param(model.plants, initialize=plants["supply_limit"])
model.b = pyo.Param(model.customers, initialize=customers.demand)

# Objective
model.total_costs = pyo.Objective(
    expr= sum(
        model.costs_per_quantity[i,j] * model.x[i,j]
        for i in model.plants 
        for j in model.customers
    ),
    sense=pyo.minimize
)

# Constraints
def supply_rule(model, i):
    return sum(model.x[i,j] for j in model.customers) <= model.a[i] 
model.supply_limit = pyo.Constraint(model.plants, rule=supply_rule)

def demand_rule(model, j):
    return sum(model.x[i,j] for i in model.plants) >= model.b[j] 
model.satisfy_demand = pyo.Constraint(model.customers, rule=demand_rule)

def transport_rule(model, i, j):
    return model.x[i, j] >= 0
model.no_negative_transport = pyo.Constraint(model.plants, model.customers, rule=transport_rule)


In [ ]:

optimizer = pyo.SolverFactory("appsi_highs")
result = optimizer.solve(model)

result.write()
print(f"total cost: {pyo.value(model.total_costs)}")



In [ ]:
# Show the transport flows
model.x.display()

Resources:
- https://de.wikipedia.org/wiki/Transportproblem

- https://en.wikipedia.org/wiki/Transportation_theory_(mathematics)